# Notebook to demonstrate Zero shot and Few shot Learning

In [1]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
from dotenv import load_dotenv
import os
from typing import Union
import sys
sys.path.append(os.path.abspath('..'))
import numpy as np
import re

In [2]:
activity_classes = ["LAYING", "SITTING", "STANDING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS"]

def get_sample_data(path, files_per_activity=-1):
    sample_data = []
    for activity_class in activity_classes:
        subpath = os.path.join(os.path.abspath('..'), path, activity_class)
        files = os.listdir(subpath)
        num_files = len(files) if files_per_activity == -1 else files_per_activity
        for file in range(num_files):
            sample_data.append(os.path.join(subpath, files[file]))
    return sample_data

def load_csv_files(directory, label, add_timestamp=True, add_total_acc=True, trim_data=True):
    df_list = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            df['subject'] = filename.replace('.csv', '')
            # 50 samples per second
            # 1 sample takes 1/50 seconds
            if add_timestamp:
                df['timestamp'] = (df.index + 1) / 50
            if add_total_acc:
                df['total_acc'] = df['accx'] ** 2 + df['accy'] ** 2 + df['accz'] ** 2
            if trim_data:
                df_list.append(df.iloc[100:600, :])
            else:
                df_list.append(df)
    combined_df = pd.concat(df_list, ignore_index=True)
    combined_df['y'] = label
    return combined_df

def prepare_dataset(path):
    train_path = os.path.join(path, "Train")
    test_path = os.path.join(path, "Test")
    
    train_dfs = pd.concat([load_csv_files(os.path.join(train_path, activity_class), label) for label, activity_class in enumerate(activity_classes)])
    test_dfs = pd.concat([load_csv_files(os.path.join(test_path, activity_class), label) for label, activity_class in enumerate(activity_classes)])
    
    return train_dfs, test_dfs

def get_tsfel_features(tsfel_features_path):   
    data = pd.DataFrame()
    for label, activity_class in enumerate(activity_classes):
        folder_path = os.path.join(tsfel_features_path, activity_class)
        dfs = load_csv_files(folder_path, label, False, False, False)
        data = pd.concat([data, dfs], axis=0, ignore_index=True)
    return data

In [3]:
class Example:
    def __init__(self, data: Union[pd.DataFrame, None], classification: Union[str, None], modify=False, precision=None):
        self.data = self.modify_data(data, precision) if modify else data
        self.classification = classification
        self.text = self.format_data_to_string(self.data)

    @staticmethod
    def modify_data(data: Union[pd.DataFrame, None], precision=None):
        data = data.round(precision if precision is not None else 0) if data is not None else data
        return data
    
    @staticmethod
    def format_data_to_string(data: Union[pd.DataFrame, None]) -> str:
        if data is not None:
            array = data.to_numpy()
            array_str = np.array2string(array, separator=',', threshold=np.inf, max_line_width=np.inf)
            formatted_str = re.sub(r'[\[\]]', '', array_str).replace(' ', '')
            return formatted_str
        return ''

In [4]:
titles = [activity.title().replace('_', ' ') for activity in activity_classes]

In [5]:
load_dotenv()
groq_models = {"llama-3.1-8b-instant": "llama-3.1-8b-instant", "llama-3.1-70b-versatile": "llama-3.1-70b-versatile", "llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

In [6]:
from GenerateDataset import generate_dataset
from MakeHARdataset import X_train, X_test, y_train, y_test

def clean_data(folder: str = 'Train'):
    dataset_features = generate_dataset(filepath=r"..\Datasets\UCI HAR Dataset", folder=folder).columns
    features = ['tBodyAcc_mean()_X', 'tBodyAcc_mean()_Y', 'tBodyAcc_mean()_Z', 'tGravityAcc_mean()_X', 'tGravityAcc_mean()_Y', 'tGravityAcc_mean()_Z', 'tBodyGyro_mean()_X', 'tBodyGyro_mean()_Y', 'tBodyGyro_mean()_Z', 'y']

    X = X_train if folder != 'Test' else X_test
    y = y_train if folder != 'Test' else y_test

    dataframes = [pd.DataFrame(dataframe, columns=dataset_features[:-1]).assign(y=label) for dataframe, label in zip(X, y)]
    labels = []
    np.random.seed = 101
    np.random.shuffle(dataframes)

    for i in range(len(dataframes)):
        labels.append(titles[int(dataframes[i].iloc[0, -1])])
        dataframes[i] = dataframes[i][features[:-1]]
        dataframes[i] = dataframes[i].iloc[10:20, :]

    return dataframes, labels

train_dataframes, train_labels = clean_data('Train')
test_dataframes, test_labels = clean_data('Test')

train_examples = [Example(dataframe, label, True, 3) for dataframe, label in zip(train_dataframes, train_labels)]
test_examples = [Example(dataframe, label, True, 3) for dataframe, label in zip(test_dataframes, test_labels)]

Loading feature data from d:\IITGN\Courses\3rd sem\ES 335\es335-24-fall-assignment-1\Datasets\UCI HAR Dataset\train...
Loading feature data from d:\IITGN\Courses\3rd sem\ES 335\es335-24-fall-assignment-1\Datasets\UCI HAR Dataset\test...


# Zero Shot

In [7]:
num_api_keys = 7
model_name = "llama-3.1-70b-versatile"

In [17]:
cur_api_key = 1

query_str = """You are given some values of the following features of a human, in order:
1. tBodyAcc-mean()-X: Mean of the body acceleration signal in the X-axis (horizontal movement).
2. tBodyAcc-mean()-Y: Mean of the body acceleration signal in the Y-axis (vertical movement).
3. tBodyAcc-mean()-Z: Mean of the body acceleration signal in the Z-axis (lateral movement).
4. tGravityAcc-mean()-X: Mean of the gravity acceleration signal in the X-axis.
5. tGravityAcc-mean()-Y: Mean of the gravity acceleration signal in the Y-axis.
6. tGravityAcc-mean()-Z: Mean of the gravity acceleration signal in the Z-axis.
7. tBodyGyro_mean()_X: Mean of the body gyro signal in the X-axis.
8. tBodyGyro_mean()_Y: Mean of the body gyro signal in the Y-axis.
9. tBodyGyro_mean()_Z: Mean of the body gyro signal in the Z-axis.

Now classify the below activities into one of these activities: Laying, Sitting, Standing, Walking, Walking Downstairs, Walking Upstairs. Make sure you limit your response for each activity to ONLY your classification, without providing any reasoning or justification for it. If you're not sure, return your best guess:

"""

test_template = """
=======================================================
Activity {activity_number}:

{activity}

"""

for cur_example, example in enumerate(test_examples):
    query_str += test_template.format(activity_number=cur_example + 1, activity=example.text)

response = None
while not response:
    try:
        llm = ChatGroq(model=groq_models[model_name], api_key=os.environ[f'API_KEY_{cur_api_key}'], temperature=0)
        response = llm.invoke(query_str).content.strip()
    except:
        cur_api_key = (cur_api_key % num_api_keys) + 1

In [18]:
classifications = response.split('\n')[2:]
classifications = [classification[classification.index('.')+2:] for classification in classifications]

correct = sum(1 if classification == test_labels[i] else 0 for i, classification in enumerate(classifications))
total = 54
accuracy = correct / total

accuracy *= 100
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 20.37%


In [ ]:
query_str = """You are given some values of the following features of a human, in order:
1. tBodyAcc-mean()-X: Mean of the body acceleration signal in the X-axis (horizontal movement).
2. tBodyAcc-mean()-Y: Mean of the body acceleration signal in the Y-axis (vertical movement).
3. tBodyAcc-mean()-Z: Mean of the body acceleration signal in the Z-axis (lateral movement).
4. tGravityAcc-mean()-X: Mean of the gravity acceleration signal in the X-axis.
5. tGravityAcc-mean()-Y: Mean of the gravity acceleration signal in the Y-axis.
6. tGravityAcc-mean()-Z: Mean of the gravity acceleration signal in the Z-axis.
7. tBodyGyro_mean()_X: Mean of the body gyro signal in the X-axis.
8. tBodyGyro_mean()_Y: Mean of the body gyro signal in the Y-axis.
9. tBodyGyro_mean()_Z: Mean of the body gyro signal in the Z-axis.

Your task is to classify the activity into one of the following classes: Laying, Sitting, Standing, Walking, Walking Downstairs, Walking Upstairs.

Now classify the below activity. Make sure you limit your response to ONLY your classification, without providing any reasoning or justification for it. If you're not sure, return your best guess:

"""

correct = 0
total = 0
cur_example = 0
cur_api_key = 1

while cur_example < len(test_examples):
    try:
        llm = ChatGroq(model=groq_models[model_name], api_key=os.environ[f'API_KEY_{cur_api_key}'], temperature=0)
        response = llm.invoke(query_str + test_examples[cur_example].text).content.strip()
        print(response)
        print(test_examples[cur_example].classification)
        print()

        if response == test_examples[cur_example].classification:
            correct += 1
        total += 1
        cur_example += 1
    except:
        print(f'API key changed to {cur_api_key % num_api_keys + 1}')
        cur_api_key = (cur_api_key % num_api_keys) + 1 

accuracy = correct / total
accuracy *= 100
print(f'Accuracy: {accuracy:.2f}%')

# Few Shot 

## Passing all tests together

In [ ]:
model_name = "llama-3.1-70b-versatile"
cur_api_key = 1

In [ ]:
query_str = """You are given some values of the following features of a human, in order:
1. tBodyAcc-mean()-X: Mean of the body acceleration signal in the X-axis (horizontal movement).
2. tBodyAcc-mean()-Y: Mean of the body acceleration signal in the Y-axis (vertical movement).
3. tBodyAcc-mean()-Z: Mean of the body acceleration signal in the Z-axis (lateral movement).
4. tGravityAcc-mean()-X: Mean of the gravity acceleration signal in the X-axis.
5. tGravityAcc-mean()-Y: Mean of the gravity acceleration signal in the Y-axis.
6. tGravityAcc-mean()-Z: Mean of the gravity acceleration signal in the Z-axis.
7. tBodyGyro_mean()_X: Mean of the body gyro signal in the X-axis.
8. tBodyGyro_mean()_Y: Mean of the body gyro signal in the Y-axis.
9. tBodyGyro_mean()_Z: Mean of the body gyro signal in the Z-axis.

Your task is to classify the activity into one of the following classes: Laying, Sitting, Standing, Walking, Walking Downstairs, Walking Upstairs.

Here are some examples to help you get started:
"""

example_template = """
=======================================================
- Activity:
{activity}

- Classification: {classification}
"""

for example in train_examples:
    query_str += example_template.format(activity=example.text, classification=example.classification)

ending_str = """
Now classify the below activities. Make sure you limit your response for each activity to ONLY your classification, without providing any reasoning or justification for it. If you're not sure, return your best guess:
"""

query_str += ending_str

test_template = """
=======================================================
Activity {activity_number}:

{activity}

"""

for cur_example, example in enumerate(test_examples):
    query_str += test_template.format(activity_number=cur_example + 1, activity=example.text)

response = None
while not response:
    try:
        llm = ChatGroq(model=groq_models[model_name], api_key=os.environ[f'API_KEY_{cur_api_key}'], temperature=0)
        response = llm.invoke(query_str).content.strip()
    except:
        cur_api_key = (cur_api_key % 5) + 1

In [ ]:
classifications = response.split('\n\n')
classifications = [classification[classification.index(':')+2:] for classification in classifications]

correct = sum(1 if classification == test_labels[i] else 0 for i, classification in enumerate(classifications))
total = 54
accuracy = correct / total

accuracy *= 100
print(f'Accuracy: {accuracy:.2f}%')

## Passing tests one-by-one

In [21]:
query_str = """You are given some values of the following features of a human, in order:
1. tBodyAcc-mean()-X: Mean of the body acceleration signal in the X-axis (horizontal movement).
2. tBodyAcc-mean()-Y: Mean of the body acceleration signal in the Y-axis (vertical movement).
3. tBodyAcc-mean()-Z: Mean of the body acceleration signal in the Z-axis (lateral movement).
4. tGravityAcc-mean()-X: Mean of the gravity acceleration signal in the X-axis.
5. tGravityAcc-mean()-Y: Mean of the gravity acceleration signal in the Y-axis.
6. tGravityAcc-mean()-Z: Mean of the gravity acceleration signal in the Z-axis.
7. tBodyGyro_mean()_X: Mean of the body gyro signal in the X-axis.
8. tBodyGyro_mean()_Y: Mean of the body gyro signal in the Y-axis.
9. tBodyGyro_mean()_Z: Mean of the body gyro signal in the Z-axis.

Your task is to classify the activity into one of the following classes: Laying, Sitting, Standing, Walking, Walking Downstairs, Walking Upstairs.

Here are some examples to help you get started:
"""

example_template = """
=======================================================
- Activity:
{activity}

- Classification: {classification}
"""

for example in train_examples:
    query_str += example_template.format(activity=example.text, classification=example.classification)

ending_str = """
Now classify the below activity. Make sure you limit your response to ONLY your classification, without providing any reasoning or justification for it. If you're not sure, return your best guess. Also, do not add any punctuations. Limit your response to the exact classification and nothing else:

"""

query_str += ending_str

correct = 0
total = 0
cur_example = 0
cur_api_key = 1

while cur_example < len(test_examples):
    try:
        llm = ChatGroq(model=groq_models[model_name], api_key=os.environ[f'API_KEY_{cur_api_key}'], temperature=0)
        response = llm.invoke(query_str + test_examples[cur_example].text).content.strip()
        print(response)
        print(test_examples[cur_example].classification)
        print()

        if response == test_examples[cur_example].classification:
            correct += 1
        total += 1
        cur_example += 1
    except:
        cur_api_key = (cur_api_key % num_api_keys) + 1 

accuracy = correct / total
accuracy *= 100
print(f'Accuracy: {accuracy:.2f}%')

Sitting
Laying

Sitting
Sitting

Standing
Laying

Walking Upstairs
Walking Upstairs

Walking Downstairs
Walking Downstairs

Laying
Walking

Walking
Walking

Sitting
Standing

Sitting
Sitting

Walking
Walking

Laying
Laying

Walking Downstairs
Walking Downstairs

Walking Downstairs
Walking Downstairs

Laying
Walking Downstairs

Sitting
Laying

Laying
Walking

Sitting
Standing

Walking Downstairs
Laying

Sitting
Sitting

Standing
Standing

Walking Upstairs
Walking Upstairs

Sitting
Standing

Walking
Walking Downstairs

Sitting
Walking Upstairs

Walking Upstairs
Walking Upstairs

Laying
Laying

Standing
Laying

Standing
Sitting

Walking Downstairs
Sitting

Walking Downstairs
Walking Downstairs

Walking Downstairs
Walking Downstairs

Standing
Standing

Standing
Standing

Standing
Standing

Sitting
Sitting

Walking Downstairs
Walking Downstairs

Sitting
Sitting

Walking Upstairs
Walking Upstairs

Standing
Laying

Walking
Walking

Walking
Walking Upstairs

Standing
Standing

Standing
Standin

# 3. What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data?

There are many limitations of using Large Language Models (LLMs) in general, especially for tasks such as classifying human activities based on featureized accelerometer data. The following are some common issues:

- **Pretrained knowledge**: These approaches rely heavily on pretrained knowledge to generate new tokens. While Few-Shot Learning is capable of "learning" from new context, it still does not perform well on niche, domain-specific tasks like human activity classification with featurized accelerometer data due to the lack of such examples in training. This is why models are first fine-tuned on custom datasets beforehand for tasks like this. This also helps eliminate the inherent bias associated with the task in the already existing model.

- **Scalability**: The scalability of using LLMs for classification tasks like this is bottlenecked by the context window of the LLM. In our case, Groq primarily has LLMs with a context window of size 8192. Mixtral had a slightly more generous limit of 32,768 tokens. Fortunately, Meta's new LLM - Llama 3.1 had the largest context window of 131,072 tokens. This turned out to be very useful in our case, especially for Few-Shot learning, as we could pass in more examples. However, using more tokens increases the time taken to process a prompt. 

- **Use Case**: LLMs are excellent in capturing semantic relationships between text. However, they might not perform as expected in tasks such as ours. One particularly famous and recent example is the question "Which number is greater: 9.9 or 9.11?" Llama 3 answered 9.11 which is incorrect. One probable reason for this is that numbers like 9.9 and 9.11 occur frequently in sites like GitHub and denote version numbers, and indeed, 9.11 is greater than 9.9 in such cases. Such results depend on the tokenizer used by the LLM. Llama 3's tokenizer treats "11" and "9" as single tokens, so it makes sense to say that 9.11 is greater than 9.9. This makes the approach of using LLMs ill-suited for our task as we work with data that is time-series, highly featurized, and very numerical.

Now for comparing the limitations of Zero-Shot Learning and Few-Shot Learning:

- **Zero-Shot Learning**
1. It performs poorly when the activities are dissimilar from the pretrained training data of the model or when the features lack any meanigful semantic relationship, like in the case of featurized accelerometer data.
2. It often results in lower accuracy when compared to fully supervised methods. This can be demonstrated empirically as well by confirming that Few-Shot Learning and decision tree performed much better than it.

- **Few-Shot Learning**
1. Few-Shot learning methods can often lead to overfitting. While experimenting, we found that it incorrectly predicted 100% of values correctly because we did not shuffle the data. We suspect that it was because we gave the data in order: Laying, Sitting, Standing, Walking, Walking Downstairs, Walking Upstairs repeatedly.
2. While passing many examples may aid the model in capturing more information, it uses many tokens. We were forced to use the model with the highest context window of 131,072 tokens to fit in all the examples. This is computationally expensive, both, in terms of cost and performance.

# 5. Test the model with random data (ensuring the data has the same dimensions and range as the previous input) and report the results.

In [8]:
random_data = []
np.random.seed = 101

for i in range(10):
    df = pd.DataFrame(np.random.uniform(low=-1, high=1, size=(10, 9)), columns=['tBodyAcc_mean()_X', 'tBodyAcc_mean()_Y', 'tBodyAcc_mean()_Z', 'tGravityAcc_mean()_X', 'tGravityAcc_mean()_Y', 'tGravityAcc_mean()_Z', 'tBodyGyro_mean()_X', 'tBodyGyro_mean()_Y', 'tBodyGyro_mean()_Z'])
    random_data.append(Example(df, None, True, 3))

In [9]:
# testing with zero-shot learning
cur_api_key = 1
model_name = "gemma-9b"

query_str = """You are given some values of the following features of a human, in order:
1. tBodyAcc-mean()-X: Mean of the body acceleration signal in the X-axis (horizontal movement).
2. tBodyAcc-mean()-Y: Mean of the body acceleration signal in the Y-axis (vertical movement).
3. tBodyAcc-mean()-Z: Mean of the body acceleration signal in the Z-axis (lateral movement).
4. tGravityAcc-mean()-X: Mean of the gravity acceleration signal in the X-axis.
5. tGravityAcc-mean()-Y: Mean of the gravity acceleration signal in the Y-axis.
6. tGravityAcc-mean()-Z: Mean of the gravity acceleration signal in the Z-axis.
7. tBodyGyro_mean()_X: Mean of the body gyro signal in the X-axis.
8. tBodyGyro_mean()_Y: Mean of the body gyro signal in the Y-axis.
9. tBodyGyro_mean()_Z: Mean of the body gyro signal in the Z-axis.

Now classify the below activities into one of these activities: Laying, Sitting, Standing, Walking, Walking Downstairs, Walking Upstairs. Make sure you limit your response for each activity to ONLY your classification, without providing any reasoning or justification for it. If you're not sure, return your best guess:

"""

test_template = """
=======================================================
Activity {activity_number}:

{activity}

"""

for cur_example, example in enumerate(random_data):
    query_str += test_template.format(activity_number=cur_example + 1, activity=example.text)

In [15]:
llm = ChatGroq(model=groq_models['mixtral'], api_key="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", temperature=0)
llm.invoke('Hi')

AIMessage(content="Hello! How can I help you today? If you have any questions about a specific topic, feel free to ask. I'll do my best to provide a clear and concise answer. If you're just looking for someone to chat with, I'm here for that too. Is there something on your mind that you'd like to talk about? I'm all ears.", response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 9, 'total_tokens': 90, 'completion_time': 0.129655201, 'prompt_time': 0.001822658, 'queue_time': 0.012468221, 'total_time': 0.131477859}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-55e0cd72-d21f-486c-912a-6fd3a5724160-0', usage_metadata={'input_tokens': 9, 'output_tokens': 81, 'total_tokens': 90})

In [11]:
response = None
while not response:
    try:
        llm = ChatGroq(model=groq_models[model_name], api_key=os.environ[f'API_KEY_{cur_api_key}'], temperature=0)
        response = llm.invoke(query_str).content.strip()
    except:
        cur_api_key = (cur_api_key % num_api_keys) + 1

"You are given some values of the following features of a human, in order:\n1. tBodyAcc-mean()-X: Mean of the body acceleration signal in the X-axis (horizontal movement).\n2. tBodyAcc-mean()-Y: Mean of the body acceleration signal in the Y-axis (vertical movement).\n3. tBodyAcc-mean()-Z: Mean of the body acceleration signal in the Z-axis (lateral movement).\n4. tGravityAcc-mean()-X: Mean of the gravity acceleration signal in the X-axis.\n5. tGravityAcc-mean()-Y: Mean of the gravity acceleration signal in the Y-axis.\n6. tGravityAcc-mean()-Z: Mean of the gravity acceleration signal in the Z-axis.\n7. tBodyGyro_mean()_X: Mean of the body gyro signal in the X-axis.\n8. tBodyGyro_mean()_Y: Mean of the body gyro signal in the Y-axis.\n9. tBodyGyro_mean()_Z: Mean of the body gyro signal in the Z-axis.\n\nNow classify the below activities into one of these activities: Laying, Sitting, Standing, Walking, Walking Downstairs, Walking Upstairs. Make sure you limit your response for each activity